In [ ]:
import os
import sys
import time

verbose  = os.environ.get('RADICAL_PILOT_VERBOSE', 'REPORT')
os.environ['RADICAL_PILOT_VERBOSE'] = verbose

import radical.pilot as rp
import radical.utils as ru

from rose.learner import ReinforcementLearner
from rose.engine import Task, ResourceEngine

In [ ]:
engine = ResourceEngine({'runtime': 30,
                         'resource': 'local.localhost'})
rl = ReinforcementLearner(engine)
code_path = f'{sys.executable} {os.getcwd()}'

In [ ]:
# Define and register the environment task
@rl.environment_task
def environment(*args):
    return Task(executable=f'{code_path}/environment.py', arguments=args)

# Define and register the policy update task
@rl.update_task
def update(*args):
    return Task(executable=f'{code_path}/update.py', arguments=args)


In [ ]:
# Custom teaching loop with reinforcement Learning
def teach():
    env = environment()
    env.result()
    upd = update()
    upd.result()
# invoke the custom/user-defined teach() method
teach()

In [ ]:
engine.shutdown()